In [36]:
import numpy as np

In [104]:
def calculate_performance_loss(baseline_acc, current_acc):
    """Calculate normalized performance loss."""
    return (baseline_acc - current_acc) / baseline_acc if baseline_acc != 0 else 0

def calculate_differential_losses(pruning_results, baseline):
    """
    Calculate the differential of performance losses across pruning ratios.
    Arguments:
    pruning_results -- list of tuples, where each tuple contains (pruning_ratio, obj_acc, rel_acc, triplet_acc)
    baseline -- tuple of (baseline_obj_acc, baseline_rel_acc, baseline_triplet_acc)
    """
    # Sorting by pruning ratios for accurate differential calculation
    sorted_results = sorted(pruning_results, key=lambda x: x[0])
    ratios = [x[0] for x in sorted_results]
    losses = [[calculate_performance_loss(x[i + 1], baseline[i]) for i in range(3)] for x in sorted_results]
    
    # Calculating differentials of losses
    diffs = np.diff(np.array(losses), axis=0)
    diffs = np.vstack([diffs, diffs[-1]])  # Duplicate last differential as an approximation for the last point
    return ratios, losses, diffs

def calculate_lambda(losses):
    """Calculate dynamic lambda value based on the range and max differential of losses."""
    if losses.size == 0:
        return np.array([1, 1, 1])  # 임시 기본값을 반환
    loss_ranges = np.ptp(losses, axis=0)  # Range of each metric's losses
    diffs = np.diff(losses, axis=0)
    if diffs.size == 0:
        max_diffs = np.array([1, 1, 1])  # 임시 기본값을 반환
    else:
        max_diffs = np.max(np.abs(diffs), axis=0)  # Max differential of each metric's losses
    lambda_values = loss_ranges / max_diffs  # Calculate lambda for each metric
    return lambda_values


def calculate_model_score(losses, diffs, weights=(1, 1, 5), scaling_factor=5.0, mu=0.1, pruning_ratio=0):
    """Calculate scores incorporating the absolute values of losses and their differentials, adjusted by lambda."""
    lambda_values = calculate_lambda(np.array([losses])) * scaling_factor  # Convert losses to array and apply scaling factor
    adjusted_losses = np.abs(losses)
    adjusted_diffs = np.abs(diffs)
    
    # Calculate score by combining losses, diffs, and the pruning ratio impact
    score = sum(weights[i] * (adjusted_losses[i] - lambda_values[i] * adjusted_diffs[i]) for i in range(3)) + mu * pruning_ratio
    return score

In [105]:

sgpn_pruning_results = [
    (0.75, 29.989, 79.029, 76.868),
    (0.55, 22.592, 77.589, 75.373),
    (0.6, 24.510, 78.692, 76.251),
    (0.3, 42.276, 81.235, 83.209),
    (0.7, 30.074, 78.650, 76.759),
    (0.35, 38.377, 81.069, 81.855),
    (0.2, 46.491, 82.212, 84.951),
    (0.25, 45.817, 81.486, 84.443),
    (0.1, 48.957, 82.641, 85.769),
    (0.4, 30.285, 78.940, 79.357),
    (0.15, 47.524, 82.594, 85.095),
    (0.05, 49.989, 82.500, 85.943),
    (0.65, 31.191, 79.042, 78.219),
    (0.5, 33.867, 77.175, 77.054),
    (0.45, 33.783, 78.675, 79.545)
]
# SGPN 모델의 Baseline 성능
sgpn_baseline = (50.158, 82.324, 85.980)

# SGFN 모델의 unstructured pruning 결과
sgfn_pruning_results = [
   (0.65, 49.631, 89.071, 87.256),
    (0.75, 43.267, 87.398, 84.398),
    (0.45, 51.949, 89.807, 88.417),
    (0.5, 51.823, 89.681, 88.208),
    (0.35, 52.055, 89.879, 88.528),
    (0.3, 52.308, 89.998, 88.506),
    (0.25, 52.624, 89.986, 88.491),
    (0.6, 50.896, 89.267, 87.631),
    (0.05, 52.476, 89.956, 88.508),
    (0.7, 46.575, 88.337, 85.920),
    (0.1, 52.476, 89.946, 88.516),
    (0.4, 51.970, 89.874, 88.508),
    (0.15, 52.497, 89.973, 88.508),
    (0.2, 52.518, 89.906, 88.513),
    (0.55, 51.149, 89.525, 87.928)
]

# SGFN 모델의 Baseline 성능
sgfn_baseline = (52.476, 89.968, 88.486)

# Attn + SGFN 모델의 unstructured pruning 결과
attn_sgfn_pruning_results = [
    (0.2, 54.499, 88.620, 89.066),
    (0.45, 51.886, 88.322, 88.030),
    (0.6, 45.838, 88.412, 85.464),
    (0.55, 49.463, 88.196, 87.147),
    (0.5, 50.664, 88.186, 87.576),
    (0.7, 43.077, 87.576, 84.755),
    (0.15, 54.626, 88.632, 89.106),
    (0.4, 52.750, 88.498, 88.417),
    (0.35, 53.678, 88.707, 88.655),
    (0.75, 36.164, 86.476, 81.444),
    (0.3, 53.656, 88.548, 88.689),
    (0.05, 54.478, 88.764, 89.187),
    (0.25, 54.542, 88.751, 88.979),
    (0.1, 54.457, 88.677, 89.195),
    (0.65, 43.498, 87.435, 84.921)
]

# Attn + SGFN 모델의 Baseline 성능
attn_sgfn_baseline = (54.499, 88.734, 89.180)

# VLSAT 모델의 unstructured pruning 결과
vlsat_pruning_results = [
    (0.5, 54.394, 88.565, 88.843),
    (0.65, 51.570, 88.959, 87.879),
    (0.4, 54.689, 88.776, 89.123),
    (0.3, 54.689, 89.078, 89.215),
    (0.15, 54.773, 88.778, 89.247),
    (0.35, 54.605, 89.006, 89.277),
    (0.75, 46.175, 87.631, 84.931),
    (0.25, 54.689, 88.746, 89.259),
    (0.1, 54.773, 88.788, 89.314),
    (0.55, 53.846, 88.739, 88.471),
    (0.45, 54.858, 88.816, 88.984),
    (0.05, 54.731, 88.751, 89.344),
    (0.2, 55.047, 88.761, 89.304),
    (0.7, 47.334, 88.094, 85.801),
    (0.6, 53.256, 88.756, 88.236)
]

# VLSAT 모델의 Baseline 성능
vlsat_baseline = (54.837, 88.766, 89.336)

In [106]:
models_data = {
    "SGPN": {
        "baseline": sgpn_baseline,
        "pruning_results": sgpn_pruning_results
    },
    "SGFN": {
        "baseline": sgfn_baseline,
        "pruning_results": sgfn_pruning_results
    },
    "Attn + SGFN": {
        "baseline": attn_sgfn_baseline,
        "pruning_results": attn_sgfn_pruning_results
    },
    "VLSAT": {
        "baseline": vlsat_baseline,
        "pruning_results": vlsat_pruning_results
    }
}

In [111]:
def display_scores(models_data, weights=(1, 2, 5), scaling_factor=3.0, mu=5):
    results = {}
    for model_name, data in models_data.items():
        baseline = data["baseline"]
        pruning_results = data["pruning_results"]
        if not pruning_results or len(pruning_results) < 2:
            print(f"Not enough data to calculate scores for {model_name}.")
            continue
        ratios, losses, diffs = calculate_differential_losses(pruning_results, baseline)
        
        scores = [calculate_model_score(losses[idx], diffs[idx], weights, scaling_factor, mu, ratios[idx])
                  for idx in range(len(ratios))]
        
        max_score_index = np.argmax(scores)
        best_ratio = ratios[max_score_index]
        best_score = scores[max_score_index]

        results[model_name] = {
            "best_ratio": best_ratio,
            "best_score": best_score,
            "all_scores": scores
        }

        print(f"Scores for {model_name}:")
        for ratio, score in zip(ratios, scores):
            print(f"  Pruning Ratio {ratio}, Score: {score:.3f}")
        print(f"Best Pruning Ratio: {best_ratio}, Highest Score: {best_score:.3f}\n")

    return results

In [112]:
final_results = display_scores(models_data)
#final_results

Scores for SGPN:
  Pruning Ratio 0.05, Score: 0.015
  Pruning Ratio 0.1, Score: 0.055
  Pruning Ratio 0.15, Score: 0.129
  Pruning Ratio 0.2, Score: 0.162
  Pruning Ratio 0.25, Score: 0.231
  Pruning Ratio 0.3, Score: 0.410
  Pruning Ratio 0.35, Score: 0.625
  Pruning Ratio 0.4, Score: 1.199
  Pruning Ratio 0.45, Score: 1.027
  Pruning Ratio 0.5, Score: 1.244
  Pruning Ratio 0.55, Score: 2.101
  Pruning Ratio 0.6, Score: 1.837
  Pruning Ratio 0.65, Score: 1.252
  Pruning Ratio 0.7, Score: 1.432
  Pruning Ratio 0.75, Score: 1.424
Best Pruning Ratio: 0.55, Highest Score: 2.101

Scores for SGFN:
  Pruning Ratio 0.05, Score: 0.007
  Pruning Ratio 0.1, Score: 0.012
  Pruning Ratio 0.15, Score: 0.017
  Pruning Ratio 0.2, Score: 0.024
  Pruning Ratio 0.25, Score: 0.028
  Pruning Ratio 0.3, Score: 0.035
  Pruning Ratio 0.35, Score: 0.047
  Pruning Ratio 0.4, Score: 0.053
  Pruning Ratio 0.45, Score: 0.063
  Pruning Ratio 0.5, Score: 0.085
  Pruning Ratio 0.55, Score: 0.123
  Pruning Ratio 0.6,